In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('company_nps_data.csv',encoding='cp949')

In [5]:
data.head()

,회사ID,연매출액,년도,월,월별_연금보험료,월별_직원수,연월
0,233757,41688077,2018,07,8912380,36,201807
1,123464,1760695,2018,03,942380,5,201803
2,342124,3221341,2018,07,781180,4,201807
3,386420,4815584,2015,11,3795900,14,201511
4,129243,9799736,2018,10,40724680,151,201810


In [29]:
data['년도'] = data['년도'].astype(str)
data['월'] = data['월'].astype(str)
data['월'] = data['월'].str.zfill(2)

data['연월'] = data['년도'] + data['월']

In [ ]:
# Nan 값 없음
data.isna().sum()

회사ID        0
연매출액        0
년도          1
월           1
월별_연금보험료    1
월별_직원수      1
dtype: int64

3년 이상 된 기업

In [3]:
from collections import Counter

In [4]:
years = data['년도'].unique()
company = []

for year in years:
  company += list(set(data[data['년도']==year].iloc[:,0].tolist()))

In [5]:
company = Counter(company).most_common()
under_3 = [c[0]for c in company if c[1]<3]

In [6]:
df = data[~data['회사ID'].isin(under_3)]

연매출 마이너스 기업 제외

In [7]:
# 1곳 해당
temp = df[df['연매출액']<0].iloc[:,0].tolist()

In [8]:
df = df[~df['회사ID'].isin(temp)]

직원수 0 기업 제외 => 3년 이상인 기업에서는 없음 5인 미만 영세기업 제외는 어떨까? 

In [9]:
# 1곳 해당
temp = df[df['월별_직원수']==0].iloc[:,0].tolist()

In [19]:
len(set(temp))

84

In [13]:
# 331곳 해당
temp = df[df['월별_직원수']<5].iloc[:,0].tolist()

In [10]:
df = df[~df['회사ID'].isin(temp)]

연매출액 1000억 이상 기업 (연매출 단위: 천원)

In [11]:
# 84개 / 82개
temp = df[df['연매출액']>=100000000].iloc[:,0].tolist()

In [30]:
# 274개 / 264개
temp = df[df['연매출액']>=30000000].iloc[:,0].tolist()

In [12]:
df = df[df['회사ID'].isin(temp)]

보험료, 직원수, 연매출 증가율 칼럼 추가

In [13]:
# 회사ID, 년도, 월 순으로 정렬 후 인덱스 다시 매김
df = df.sort_values(by=['회사ID','년도','월'])
df = df.reset_index(drop=True)

t = df

In [19]:
t['전월_직원수'] = t.groupby('회사ID')['월별_직원수'].shift(1)
t['전월_연금보험료'] = t.groupby('회사ID')['월별_연금보험료'].shift(1)
t['전년도_매출액'] = t.groupby('회사ID')['연매출액'].shift(1)

t['직원수_증감률'] = ( (t['월별_직원수'] - t['전월_직원수']) / t['전월_직원수'] ) * 100
t['연금보험료_증감률'] = ( (t['월별_연금보험료'] - t['전월_연금보험료']) / t['전월_연금보험료'] ) * 100
t['매출액_증감률'] = ( (t['연매출액'] - t['전년도_매출액']) / t['전년도_매출액'] ) * 100

In [20]:
t

,회사ID,연매출액,년도,월,월별_연금보험료,월별_직원수,전월_직원수,전월_연금보험료,전년도_매출액,직원수_증감률(%),연금보험료_증감률(%),매출액_증감률(%),직원수_증감률,연금보험료_증감률,매출액_증감률
0,126516,1061715908,2015,11,589400960,1674,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,126516,1061715908,2015,12,589086820,1664,1674.0,589400960.0,1.061716e+09,-0.597372,-0.053298,0.000000,-0.597372,-0.053298,0.000000
2,126516,468913812,2016,1,578260840,1618,1664.0,589086820.0,1.061716e+09,-2.764423,-1.837756,-55.834342,-2.764423,-1.837756,-55.834342
3,126516,468913812,2016,2,581872420,1635,1618.0,578260840.0,4.689138e+08,1.050680,0.624559,0.000000,1.050680,0.624559,0.000000
4,126516,468913812,2016,3,579106460,1623,1635.0,581872420.0,4.689138e+08,-0.733945,-0.475355,0.000000,-0.733945,-0.475355,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3093,510329,143513683,2018,8,137202420,609,576.0,132073100.0,1.435137e+08,5.729167,3.883698,0.000000,5.729167,3.883698,0.000000
3094,510329,143513683,2018,9,137436680,609,609.0,137202420.0,1.435137e+08,0.000000,0.170740,0.000000,0.000000,0.170740,0.000000
3095,510329,143513683,2018,10,138112640,614,609.0,137436680.0,1.435137e+08,0.821018,0.491834,0.000000,0.821018,0.491834,0.000000
3096,510329,143513683,2018,11,136699180,606,614.0,138112640.0,1.435137e+08,-1.302932,-1.023411,0.000000,-1.302932,-1.023411,0.000000


In [30]:
#전년도 대비 매출액이 한 번이라도 작았던 기업 => 56 곳 
temp = t[t['매출액_증감률']<0].iloc[:,0].tolist()